In [16]:
# this is web scraping for yahoo articles that have the term "ai" in it
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

# make this for da url template
template = 'https://news.search.yahoo.com/search?p={}'
headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

In [17]:
# this is for extracting articles
def get_article(card):
    headline = card.find('h4', 's-title').text
    source = card.find('span', 's-source').text
    posted = card.find('span', 's-time').text.replace('.', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clear_link = re.search(pattern, unquoted_link).group(1)
    article = (headline, source, posted, description, clear_link)
    return article

In [18]:
# this is the function to scrape Yahoo news for articles for ai
def get_ai_articles():
    articles = []
    links = set()
    search_query = 'AI'  # Change this to the keyword you want to search for

    url = template.format(search_query)

    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        cards = soup.find_all('div', 'NewsArticle')
    
    for card in cards:
        article = get_article(card)
        link = article[-1]

        if link not in links:
            links.add(link)
            articles.append(article)
            
        try:
            url = soup.find('a', 'next').get('href')
            sleep(2)  # Sleep to avoid being seen as a bot
        except AttributeError:
            break
            
        with open('ai_articles.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Headline', 'Source', 'Posted', 'Description', 'Link'])
            writer.writerows(articles)

        return articles

In [ ]:
# scrape the yahoo news for articles with ai
ai_articles = get_ai_articles()
print(len(ai_articles))  # print num of articles collected

In [ ]:
#articles = get_the_news('brexit')

In [ ]:
#print(len(articles)) # 335